In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_models.llamacpp import ChatLlamaCpp
from langchain.prompts import PromptTemplate
import warnings

warnings.filterwarnings("ignore")

In [ ]:
llm = ChatLlamaCpp(
    model_path="./tinyllama-1.1b-chat-v1.0.Q5_K_M.gguf",
    stop=['User', '\nUser','\n\n', '\nHuman', "</s>", " Human:"],
    temperature=0.8,
    streaming=True,
    max_tokens=256,
    top_p=0.9,
    verbose=False,
    n_ctx=2048,
    n_batch=32,
)

In [ ]:
summary_prompt = PromptTemplate.from_template(
    (
        "Summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n"
        "Previous summary:\n"
        "{summary}\n\n"
        "New lines of conversation:\n"
        "{new_lines}\n\n"
        "New summary:\n"
    )
)

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100, prompt=summary_prompt)

In [ ]:
prompt = PromptTemplate.from_template(
    (
        "This is a conversation between human and AI. Reply to the user in NO MORE THAN 30 words.\n"
        "History: {history}\n"
        "Human: {input}\n\n"
        # "assistant|>"
    )
)

In [ ]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose = False
)

### Run (No streaming)

In [ ]:
chain.run("Hello.")

In [ ]:
chain.run("I will call you Jarvis.")

In [ ]:
history = chain.memory.buffer
history

In [ ]:
chain.run("What is your name?")

In [ ]:
chain.run("Good. What are you doing today?")

In [ ]:
chain.run("What is the project?")

In [ ]:
chain.memory.buffer

### Streaming

In [ ]:
memory.clear()
chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose = False
)

In [ ]:
chain.stream(input={"input": "Hello."})

In [ ]:
for token in chain.stream(input={"input": "Hello."}):
    print(token, end="", flush=True)

In [ ]:
for token in chain.stream(input={"input": "I will call you Jarvis. Ok?"}):
    print(token, end="", flush=True)

In [ ]:
for token in chain.stream(input={"input": "What is your name?"}):
    print(token, end="", flush=True)